In [ ]:
# Create a set of lists to store objects from json file
ids = []
questions = []
backgrounds = []
closeTime = []
tags = []
answers = []
choices = []
status = []
qtype = []

# Extracting data and appending to a Pandas DataFrame
f = open('autocast_questions.json', encoding="utf-8")

data = json.loads(f.read())

for jsonObj in data:
    ids.append(jsonObj["id"])
    questions.append(jsonObj["question"])
    backgrounds.append(jsonObj["background"])
    closeTime.append(jsonObj["close_time"])
    tags.append(jsonObj["tags"])
    answers.append(jsonObj["answer"])
    choices.append(jsonObj["choices"])
    status.append(jsonObj["status"])
    qtype.append(jsonObj["qtype"])
      
df = pd.DataFrame({"ID" : ids, "Question" : questions, "Background" : backgrounds, 
                   "Close_Time" : closeTime, "Tags" : tags, "Answer" : answers, 
                   "Choices" : choices, "Status" : status, "Qtype" : qtype})

f.close()

# Changing datatype to string
df["Close_Time"] = df["Close_Time"].astype("string") 

# Clearing up data in the lists after appending to a Pandas Dataframe
ids.clear()
questions.clear()
backgrounds.clear()
closeTime.clear()
tags.clear()
answers.clear()
choices.clear()
status.clear()
qtype.clear()

# Appending data before the cut-off time into the cleared lists
# Cut-off time would be March 22, 2022, 23:59:59 UTC
for i in range(len(df["Close_Time"])-1):
    if pd.to_datetime(df["Close_Time"][i], infer_datetime_format = True, errors = "coerce").tz_localize(None) < pd.Timestamp(2022, 3, 22, 23):
        ids.append(df["ID"][i])
        questions.append(df["Question"][i])
        backgrounds.append(df["Background"][i])
        tags.append(df["Tags"][i])
        answers.append(df["Answer"][i])
        choices.append(df["Choices"][i])
        status.append(df["Status"][i])
        qtype.append(df["Qtype"][i])

# Clearing the current dataFrame to store data to be use for training 
df = df.iloc[0:0] 

# Closing time is not needed anymore as we achieve the data before the cut-off time
# Plus reduces the memory needed to store for closing time
df = pd.DataFrame({"ID" : ids, "Question" : questions, "Background" : backgrounds, 
                   "Tags" : tags, "Qtype" : qtype, "Choices" : choices, "Status" : status, "Answer" : answers})

In [21]:
import json
import pandas as pd
import os
import glob

from os.path import join, dirname, abspath, exists, isdir

# relevant paths
home_path = "C:/Users/ADMIN/Desktop/Few_shot-learning"
# output_ims_dir_path = join(home_path, "data", "images")

# list of csv files in current directory
filename = "autocast_questions.csv"

# json fields
question  =  {
    'id': None,
    'question': None,
    'background' : None,
    'qtype' : None,
    'choices' : None,
    'answer' : None,
}

if __name__ == "__main__":
    # Writing to jsons
    course_name = filename[: -4].strip() # format of: "<course_code>""

    # read in sheet
    sheet = pd.read_csv(filename, encoding='utf-8')

    # set up file output directory
    out_dir_path = "C:/Users/ADMIN/Desktop/Few_shot-learning/data"
    count = -1
    
    test_question = "C:/Users/ADMIN/Desktop/Few_shot-learning/code/autocast_test_set_w_answers.xlsx"
    results = pd.read_excel(test_question)

    for i in range(len(sheet['ID'])):
        if (str(sheet['ID'][i]) in results['id'].values) or (str(sheet['Status'][i]) != 'Resolved'):
            continue
        else:
            count += 1
            question['id'] = str(sheet['ID'][i])
            question['question'] = str(sheet['Question'][i])
            question['background'] = str(sheet['Background'][i])
            question['qtype'] = str(sheet['Qtype'][i])
            question['choices'] = str(sheet['Choices'][i])
            question['answer'] = str(sheet['Answer'][i])

            json_object = json.dumps(question, indent = 7)
            fname = join(out_dir_path, 'Question'+'_'+str(sheet['ID'][i])+ '_'+str(i)+'.json')
            with open(fname, "w") as outfile:
                outfile.write(json_object)

            if str(sheet['Question_negated'][i]) != 'nan':
                question['question'] = str(sheet['Question_negated'][i])
                if question['answer'] == 'yes':
                    question['answer'] = 'no'
                else:
                    question['answer'] = 'yes'
                json_object = json.dumps(question, indent = 7)
                fname = join(out_dir_path, 'ZQuestion'+'_'+str(sheet['ID'][i])+ '_'+str(len(sheet['ID'])+count)+'.json')
                with open(fname, "w") as outfile:
                    outfile.write(json_object)
#     print(len(sheet['id'])+count)